In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt, gc, os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os
import lightgbm
import matplotlib.pyplot as plt
import seaborn as sbn
import numpy as np
import pandas as pd
import warnings
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, RocCurveDisplay, accuracy_score
import shap
import itertools

In [ ]:
train = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet").groupby('customer_ID').tail(4)
test = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet").groupby('customer_ID').tail(4)
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train.head()

In [ ]:
#shapes
train.shape, test.shape, train_labels.shape

In [ ]:
#Checking Missing values having more than 40%
columns = train.columns[(train.isna().sum()/len(train))*100>40]

In [ ]:
#Drop the missing columns having more than 40%
train = train.drop(columns, axis=1)
test = test.drop(columns, axis=1)

In [ ]:
#fill the missing values
train = train.bfill(axis='rows').ffill(axis='rows')
test = test.bfill(axis='rows').ffill(axis='rows')

In [ ]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
train =train.groupby('customer_ID').tail(1)
test = test.groupby('customer_ID').tail(1)

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
#shape
train.shape, train_labels.shape, test.shape

In [ ]:
#Type Conversion
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
for col in obj_col:
    train[col]=train[col].astype('int').astype('str')
    test[col]=test[col].astype('int').astype('str')
    print(train[col].unique())
    print(test[col].unique())

**Data Preparation**

In [ ]:
train = train.merge(train_labels, how='inner', on="customer_ID")

In [ ]:
train.head()

In [ ]:
test_data = test.copy()
train = train.drop(['index','customer_ID', 'S_2'], axis=1)
test = test.drop(['index','customer_ID', 'S_2'], axis=1)

In [ ]:
#one hot Encoding for categorical features
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
train = pd.get_dummies(train, columns=obj_col, drop_first=True)
test = pd.get_dummies(test, columns=obj_col, drop_first=True)
train.shape, test.shape

In [ ]:
Features=train.loc[:, test.columns]
target = train['target']

In [ ]:
from sklearn.preprocessing import StandardScaler
# Create an instance of the StandardScaler
scaler = StandardScaler()
cols=Features.columns
# Fit the scaler to the DataFrame
scaler.fit(Features)

# Transform the DataFrame
Features = scaler.transform(Features)
Features=pd.DataFrame(Features, columns=cols)

In [ ]:
Features.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into a training and test set
X_train, X_test, y_train, y_test = train_test_split(Features, target, test_size=0.3, random_state=0)


**Model Building and Evaluation**

   **1) SVM Classifier**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Define the hyperparameters and their possible values
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'],'gamma':[0.1,1,10, 100]}

# Create the SVM classifier
svm_clf = svm.SVC()

# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(svm_clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)
# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_

# Create a new SVM classifier with the best hyperparameters
best_svm = svm.SVC(**best_params)

# Fit the best model to the data
best_svm.fit(X, y)

In [ ]:
y_pred=best_svm.pred(X_test)

In [ ]:
cm=confusion_matrix(y_test, y_pred)
plt.figure(figsize=(13,5))
plt.title("Confusion Matrix")
plt.imshow(cm, alpha=0.5, cmap='PuBu')
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center")
plt.show()

In [ ]:
#AUC (ROC curve)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=XGB)
display.plot()
plt.show()

**2) KNN Classifier**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Define the hyperparameters and their possible values
param_grid = {'n_neighbors': [3,5,7,9,11], 'weights': ['uniform', 'distance'],'metric':['minkowski','euclidean']}

# Create the KNN classifier
knn = KNeighborsClassifier()

# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_

# Create a new KNN classifier with the best hyperparameters
best_knn = KNeighborsClassifier(**best_params)

# Fit the best model to the data
best_knn.fit(X_train, y_train)

In [ ]:
y_pred=best_knn.pred(X_test)

In [ ]:
cm=confusion_matrix(y_test, y_pred)
plt.figure(figsize=(13,5))
plt.title("Confusion Matrix")
plt.imshow(cm, alpha=0.5, cmap='PuBu')
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center")
plt.show()

In [ ]:
#AUC (ROC curve)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=XGB)
display.plot()
plt.show()

**3) XGBClassifier**

In [ ]:
XGB = XGBClassifier(n_estimators=300, max_depth=6, learning_rate=0.1).fit(Features, target)

In [ ]:
np.mean(cross_val_score(XGB, Features, target, scoring='accuracy', cv=5))

In [ ]:
y_pred.head()

In [ ]:
y_pred = XGB.predict(X_test)

In [ ]:
cm=confusion_matrix(y_test, y_pred)
plt.figure(figsize=(13,5))
plt.title("Confusion Matrix")
plt.imshow(cm, alpha=0.5, cmap='PuBu')
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center")
plt.show()

In [ ]:
#AUC (ROC curve)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=XGB)
display.plot()
plt.show()

**Submission to Kaggle**

In [ ]:
test_data['prediction']=XGB.predict_proba(test)[:,1]
test_data[['customer_ID','prediction']].to_csv("submission.csv", index=False)